In [1]:
import json
import numpy as np
import pandas as pd
from collections import Counter
import spacy
import en_core_web_sm
from spacy.symbols import nsubj, dobj, VERB
import re



In [2]:
#set-up
nlp = spacy.load("en_core_web_sm")




In [3]:
data_t_calfire = pd.read_csv('/data/zshi/test/t_results/temporal_predcalfire_calfire.csv', lineterminator='\n')
data_t_calfire


,Unnamed: 0,start_date_pred,start_date_true,title,sentence,S-N,E-N,Binary
0,0,2990-01-01,2990-01-01,Bay Area wildfire updates,While residents of Vacaville and Atlas Peak be...,Correct,Correct,Correct
1,1,2990-01-01,2990-01-01,Wildfires set school plans ablaze for more tha...,"In response , the district has ordered more ho...",Correct,Correct,Correct
2,2,2990-01-01,2990-01-01,"Crews containing Green Ridge, Frog fires dct",Fire crews continue to make progress containin...,Correct,Incorrect,Correct
3,3,2990-01-01,2990-01-01,"Evacuations lifted near California fires, some...","They died on Aug. 19 . Bone was nearly blind ,...",Correct,Correct,Correct
4,4,2990-01-01,2990-01-01,'It was so terrifying',A wildfire burning through a swath of Los Ange...,Correct,Correct,Correct
...,...,...,...,...,...,...,...,...
496,496,2990-01-01,2990-01-01,"Utility cuts power to 26,500 Californians over...",`` The reality is come midnight and through Tu...,Correct,Correct,Correct
497,497,2990-01-01,2990-01-01,"Utility cuts power to 26,500 Californians over...",San Bernardino County was expected to be the m...,Correct,Correct,Correct
498,498,2990-01-01,2990-01-01,CAL FIRE California Statewide Fire Summary for...,"October 31 , 2020 - More than 4,700 firefight...",Correct,Correct,Correct
499,499,2990-01-01,2990-01-01,Federal agencies fall short of Trump forest pr...,"In its statement , the agency said it now uses...",Correct,Correct,Correct


In [5]:
data_t_covid = pd.read_csv('/data/zshi/test/t_results/temporal_predcovid_covid.csv', lineterminator='\n')
data_t_covid



,Unnamed: 0,start_date_pred,start_date_true,title,sentence,S-N,E-N,Binary
0,0,2990-01-01,2990-01-01,"Heightened anxiety, depression test communitie...",``` Pull yourself up by your bootstraps ' is p...,Correct,Incorrect,Incorrect
1,1,2990-01-01,2990-01-01,Coronavirus In NH Schools,Community transmission outside of the classroo...,Correct,Correct,Correct
2,2,2020-12-09,2990-01-01,Allegheny County Coronavirus Death Toll Skyroc...,Here are the latest coronavirus case numbers f...,Incorrect,Correct,Correct
3,3,2990-01-01,2990-01-01,Two more COVID-19-related deaths reported in W...,"CHARLESTON , W.Va . -LRB- WCHS/WVAH -RRB- -- ...",Correct,Incorrect,Correct
4,4,2990-01-01,2990-01-01,"1st Round Pick Justin Jefferson, 3 Other Vikin...",The league 's recently-implemented reserve/COV...,Correct,Correct,Correct
...,...,...,...,...,...,...,...,...
495,495,2990-01-01,2020-10-30,Covid-19 death toll rises in Northwest Arkansa...,The four largest school districts in Northwest...,Incorrect,Correct,Correct
496,496,2990-01-01,2990-01-01,Tuesday's COVID-19 Numbers from Bergen County ...,"Statewide , there are 171,667 positive cases a...",Correct,Correct,Correct
497,497,2990-01-01,2990-01-01,"Montana Reported 50 New COVID-19 Cases, Three ...",There are currently 336 active cases in the st...,Correct,Correct,Correct
498,498,2990-01-01,2990-01-01,HEALTH DEPARTMENT UPDATES STATEWIDE COVID-19 C...,INDIANAPOLIS -- The Indiana State Department o...,Correct,Correct,Correct


In [6]:
data_t_blm = pd.read_csv('/data/zshi/test/t_results/temporal_predblm_blm.csv', lineterminator='\n')
data_t_blm



,Unnamed: 0,start_date_pred,start_date_true,title,sentence,S-N,E-N,Binary
0,0,2990-01-01,2990-01-01,Threatening email sent to NDSU President for i...,"FARGO , N.D. ( Valley News Live ) - NDSU is in...",Correct,Correct,Correct
1,1,2990-01-01,2990-01-01,Australian police arrest six at Black Lives Ma...,SYDNEY -LRB- Reuters -RRB- - Australian polic...,Correct,Incorrect,Incorrect
2,2,2990-01-01,2990-01-01,More violent clashes across the US as cops fir...,"Do n't do it ! ' And prior to that tweet , Tru...",Correct,Incorrect,Incorrect
3,3,2990-01-01,2990-01-01,Police,The actions of this person created an atmosphe...,Correct,Correct,Incorrect
4,4,2990-01-01,2990-01-01,Australian police arrest six at Black Lives Ma...,1 / 5 Australian police arrest six at Black Li...,Correct,Correct,Incorrect
...,...,...,...,...,...,...,...,...
496,496,2990-01-01,2990-01-01,"Across Southland, thousands take to streets Su...",`` A small number of units will be stationed n...,Correct,Correct,Incorrect
497,497,2990-01-01,2990-01-01,Storm drops heavy rain on George Floyd rally i...,A man in a bear costume brought a boom box and...,Correct,Correct,Incorrect
498,498,2990-01-01,2990-01-01,George Floyd live updates,The directive and a statement released by the ...,Correct,Correct,Incorrect
499,499,2990-01-01,2990-01-01,George Floyd protests have created a multicult...,"The Rev. Zach Hoover , executive director of L...",Correct,Incorrect,Incorrect


In [50]:
data_s_calfire = pd.read_csv('/data/zshi/test/s_results/spatial_pred_calfire_calfire.csv', lineterminator='\n')
data_s_calfire

,Unnamed: 0,0,1,2,3,4
0,0,California Alameda,California,Incorrect,Bay Area wildfire updates,While residents of Vacaville and Atlas Peak be...
1,1,California,California,Correct,Wildfires set school plans ablaze for more tha...,"In response , the district has ordered more ho..."
2,2,NaN,Oregon,Incorrect,"Crews containing Green Ridge, Frog fires",Fire crews continue to make progress containin...
3,3,California Santa Cruz County,California Santa Cruz,Incorrect,"Evacuations lifted near California fires, some...","They died on Aug. 19 . Bone was nearly blind ,..."
4,4,California,Oregon Los Angeles County,Incorrect,'It was so terrifying',A wildfire burning through a swath of Los Ange...
...,...,...,...,...,...,...
485,485,California,California Fresno,Incorrect,35 more people rescued from raging Creek Fire,It moved so fast that campers became trapped a...
486,486,California,California Orange County,Incorrect,Thousands without power in LA and Orange counties,"SCE , meanwhile , was making progress at resto..."
487,487,California,California,Correct,A gender reveal party caused one of several wi...,A gender reveal party reportedly started <ext...
488,488,California,California Los Angeles,Incorrect,A gender reveal party caused one of several wi...,The California Department of Forestry and Fire...


In [53]:
data_s_covid = pd.read_csv('/data/zshi/test/s_results/spatial_pred_covid_covid.csv', lineterminator='\n')
data_s_covid

,Unnamed: 0,0,1,2,3,4
0,0,Colorado,Colorado,Correct,"Heightened anxiety, depression test communities",``` Pull yourself up by your bootstraps ' is p...
1,1,New Hampshire,New Hampshire,Correct,Coronavirus In NH Schools,Community transmission outside of the classroo...
2,2,Pennsylvania Allegheny County,Pennsylvania Allegheny County,Correct,Allegheny County Coronavirus Death Toll Skyroc...,Here are the latest coronavirus case numbers f...
3,3,West Virginia,West Virginia,Correct,Two more COVID-19-related deaths reported in W...,"CHARLESTON , W.Va . -LRB- WCHS/WVAH -RRB- -- ..."
4,4,Minnesota,Minnesota Vikings,Incorrect,"1st Round Pick Justin Jefferson, 3 Other Vikin...",The league 's recently-implemented reserve/COV...
...,...,...,...,...,...,...
475,475,Arkansas,NaN,Incorrect,Record high of COVID-19 cases in Kanawha County,"Stay home when you can . If you go out , wear ..."
476,476,New Jersey,Iowa,Incorrect,Reynolds ‘pleased’ with slower rate of infecti...,"Over the past week , Iowa has averaged 2,263 n..."
477,477,Montana Missoula County,Tennessee Hampden County,Incorrect,Baker,"Constitution warship and its crew , closed in ..."
478,478,Indiana,Kansas,Incorrect,Kansas City metro adds more than 300 COVID-19 ...,The state unveiled new COVID-19 dashboards Mon...


In [54]:
data_s_blm = pd.read_csv('/data/zshi/test/s_results/spatial_pred_blm_blm.csv', lineterminator='\n')
data_s_blm

,Unnamed: 0,0,1,2,3,4
0,0,North Dakota Fargo,New York FARGO,Incorrect,Threatening email sent to NDSU President for i...,"FARGO , N.D. ( Valley News Live ) - NDSU is in..."
1,1,Minnesota Minneapolis,Oregon Portland,Incorrect,More violent clashes across the US as cops fir...,"Do n't do it ! ' And prior to that tweet , Tru..."
2,2,Minnesota Minneapolis,Minnesota Stillwater,Incorrect,Police,The actions of this person created an atmosphe...
3,3,NaN,Minnesota Stillwater,Incorrect,‘Umbrella Man’ seen on video smashing windows ...,Police have also connected the 32-year-old man...
4,4,Minnesota Minneapolis,NaN,Incorrect,Cars have hit demonstrators 104 times since Ge...,The vehicular attacks have been `` the tradema...
...,...,...,...,...,...,...
395,395,Missouri Columbia,Illinois Chicago,Incorrect,Special prosecutor tapped to take George Floyd...,Officials said protesters vandalized six Secre...
396,396,California,Washington Olympia,Incorrect,Seattle area protest updates,`` But we also will not turn away from our res...
397,397,Montana Billings,California Oakland,Incorrect,Federal Protective Service officer killed in O...,<extra_id_99> A federal law enforcement offic...
398,398,Oregon Portland,Maryland Washington,Incorrect,May 31 George Floyd protest news,`` We also had the newest cannabis dispensary ...


In [99]:
data_e_calfire = pd.read_csv('/data/zshi/test/e_results/calfire_end_to_end.csv', lineterminator='\n')
data_e_calfire

,Unnamed: 0,0,1,2,3,4,5,6
0,0,California Alameda \n,California\n,unknown 2020-08-27 \n,unknown 2020-08-27\n,Incorrect,While residents of Vacaville and Atlas Peak be...,While residents of Vacaville and Atlas Peak be...
1,1,California \n,California\n,unknown 2020-08-27 \n,unknown 2020-08-27\n,Incorrect,"In response , the district has ordered more ho...","In response , the district has ordered more ho..."
2,2,\n,Oregon\n,unknown 2020-08-26 \n,unknown 2020-08-27\n,Incorrect,Fire crews continue to make progress containin...,Fire crews continue to make progress containin...
3,3,California Santa Cruz County \n,California Santa Cruz\n,unknown 2020-08-26 \n,unknown 2020-08-26\n,Incorrect,"They died on Aug. 19 . Bone was nearly blind ,...","They died on Aug. 19 . Bone was nearly blind ,..."
4,4,California \n,Oregon Los Angeles County\n,unknown 2020-09-21 \n,unknown 2020-09-21\n,Incorrect,A wildfire burning through a swath of Los Ange...,A wildfire burning through a swath of Los Ange...
...,...,...,...,...,...,...,...,...
495,495,California \n,California\n,unknown 2020-10-25 \n,unknown 2020-10-25\n,Incorrect,`` The reality is come midnight and through Tu...,`` The reality is come midnight and through Tu...
496,496,California \n,California\n,unknown 2020-10-25 \n,unknown 2020-10-25\n,Incorrect,`` The reality is come midnight and through Tu...,`` The reality is come midnight and through Tu...
497,497,California \n,California\n,unknown 2020-10-25 \n,unknown 2020-10-25\n,Incorrect,San Bernardino County was expected to be the m...,San Bernardino County was expected to be the m...
498,498,California \n,California\n,unknown 2020-10-31 \n,unknown 2020-10-31\n,Incorrect,"October 31 , 2020 - More than 4,700 firefight...","October 31 , 2020 - More than 4,700 firefight..."


In [101]:
data_e_calfire['4'].value_counts()

Incorrect    485
Correct       15
Name: 4, dtype: int64

In [94]:
df_c = pd.DataFrame(columns = ['sentence', 'Binary', 'ents', 'locs', 'dates', 'nums', 'nouns', 'verbs'])


In [95]:
df_c

,sentence,Binary,ents,locs,dates,nums,nouns,verbs


In [96]:
for i in data_t_covid.index:
    s = data_t_covid['sentence'][i]
    s = str(s)
    s = re.sub(" <extra_id_99> ", " ", s)    
    separator = "title:"
    s = s.split(separator, 1)[0]   
    
    #print (s)
    
    doc = nlp(s)
    
    ents = [(X.text, X.label_) for X in doc.ents]
    locs = [(X.text, X.label_) for X in doc.ents if X.label_ == "GPE"]
    dates = [(X.text, X.label_) for X in doc.ents if X.label_ == "DATE"]
    nums = [(X.text, X.label_) for X in doc.ents if X.label_ == "CARDINAL"]
    nouns = [token.lemma_ for token in doc if token.pos_ == "NOUN"]
    verbs = [token.lemma_ for token in doc if token.pos_ == "VERB"]
    
    df_c = df_c.append({'sentence': data_t_covid['sentence'][i], 'Binary': data_t_covid['Binary'][i], 'ents': len(ents), 'locs': len(locs), 'dates': len(dates), 'nums': len(nums), 'nouns': len(nouns), 'verbs': len(verbs)} , ignore_index=True)
    #df_c = df_c.append({'sentence': data_s_blm['4'][i], 'city': data_s_blm['2'][i], 'ents': len(ents), 'locs': len(locs), 'dates': len(dates), 'nums': len(nums), 'nouns': len(nouns), 'verbs': len(verbs)} , ignore_index=True)
    
    
    
df_c  



,sentence,Binary,ents,locs,dates,nums,nouns,verbs
0,``` Pull yourself up by your bootstraps ' is p...,Incorrect,12,0,1,6,44,15
1,Community transmission outside of the classroo...,Correct,22,3,7,5,28,9
2,Here are the latest coronavirus case numbers f...,Correct,19,3,6,9,22,6
3,"CHARLESTON , W.Va . -LRB- WCHS/WVAH -RRB- -- ...",Correct,16,5,3,6,21,10
4,The league 's recently-implemented reserve/COV...,Correct,1,0,0,0,23,12
...,...,...,...,...,...,...,...,...
495,The four largest school districts in Northwest...,Correct,17,2,6,9,26,8
496,"Statewide , there are 171,667 positive cases a...",Correct,42,6,2,19,31,20
497,There are currently 336 active cases in the st...,Correct,16,4,1,9,23,11
498,INDIANAPOLIS -- The Indiana State Department o...,Correct,18,1,4,4,25,9


In [97]:
df_cor = df_c[df_c['Binary'] == "Correct"]
df_incor = df_c[df_c['Binary'] == "Incorrect"]
df_incor

,sentence,Binary,ents,locs,dates,nums,nouns,verbs
0,``` Pull yourself up by your bootstraps ' is p...,Incorrect,12,0,1,6,44,15
13,Hospital capacity had risen to 14 percent as o...,Incorrect,27,5,9,3,33,16
23,"Dallas County Reports 1,269 New Positive 2019...",Incorrect,22,6,3,9,21,10
51,The city 's death toll from the virus is now a...,Incorrect,23,1,13,7,35,9
80,The N.C. Department of Health and Human Servic...,Incorrect,19,2,5,7,22,15
81,"Hospitalizations , a key figure used in determ...",Incorrect,30,2,7,15,35,23
83,`` The model now predicts more firmly that in ...,Incorrect,12,4,2,3,64,28
89,`` People started talking in the spring about ...,Incorrect,13,1,5,4,25,10
102,"PORTLAND , Ore. -LRB- KOIN -RRB- -- Clark Cou...",Incorrect,11,2,2,3,14,9
117,In the nine months since the state had its fir...,Incorrect,13,0,2,4,35,20


In [98]:
print ("df_cor.ents.mean()", df_cor.ents.mean())
print ("df_cor.locs.mean()", df_cor.locs.mean())
print ("df_cor.dates.mean()", df_cor.dates.mean())
print ("df_cor.nums.mean()", df_cor.nums.mean())
print ("df_cor.nouns.mean()", df_cor.nouns.mean())
print ("df_cor.verbs.mean()", df_cor.verbs.mean())

print ("df_incor.ents.mean()", df_incor.ents.mean())
print ("df_incor.locs.mean()", df_incor.locs.mean())
print ("df_incor.dates.mean()", df_incor.dates.mean())
print ("df_incor.nums.mean()", df_incor.nums.mean())
print ("df_incor.nouns.mean()", df_incor.nouns.mean())
print ("df_incor.verbs.mean()", df_incor.verbs.mean())



df_cor.ents.mean() 24.74370709382151
df_cor.locs.mean() 4.643020594965675
df_cor.dates.mean() 4.043478260869565
df_cor.nums.mean() 9.94508009153318
df_cor.nouns.mean() 29.894736842105264
df_cor.verbs.mean() 13.183066361556063
df_incor.ents.mean() 19.714285714285715
df_incor.locs.mean() 2.5357142857142856
df_incor.dates.mean() 3.7142857142857144
df_incor.nums.mean() 7.107142857142857
df_incor.nouns.mean() 30.714285714285715
df_incor.verbs.mean() 14.285714285714286
